## Utilities

In [ ]:
from src import config

client_id, api_key, username = config.load_config()

print(f"client_id length: {len(client_id)}")
print(f"api_key length: {len(api_key)}")
print(f"username: {username}")

In [ ]:
from src.bungie_oauth import BungieAuth
import datetime

# perform oauth login to get the access token used in later requests.  It is good for 1 hour
print(
    "We're using a self-signed certificate to run an HTTPS server on localhost, you'll need to accept the certificate in your browser."
)
access_token = BungieAuth(client_id).refresh_oauth_token()

# token is good for 1 hour, print out the time that it expires
expiration_time = datetime.datetime.now() + datetime.timedelta(hours=1)
print(
    f"Access token successfully acquired at: {datetime.datetime.now().isoformat()} and expires at: {expiration_time.isoformat()}"
)

In [ ]:
# it's nice to have all of the manifest files downloaded locally into the data directory for reference
import os
import requests
from src.bungie_api import BungieApi

os.makedirs("data", exist_ok=True)

api = BungieApi(api_key, access_token)
manifest = api.get_manifest()

for key, value in manifest["Response"]["jsonWorldComponentContentPaths"]["en"].items():
    print(f"Downloading {key} from {value}")
    response = requests.get(
        f"https://www.bungie.net{value}",
        headers={
            "X-API-Key": api_key,
        },
    )
    with open(f"data/{key}.json", "w") as f:
        f.write(response.text)

### Ignore armor tagged as `infuse` or `junk` in DIM

If you [export your vault in DIM as a CSV](https://github.com/DestinyItemManager/DIM/wiki/Spreadsheets#), this can parse it and build a list of armor that the `d2armor` notebook should exclude from analysis. Adding "#ignore" to a DIM item's Notes field has the same effect — this can be used to exclude things like raid class items, which aren't junk, but are irrelevant to outfit calculations. 

In [ ]:
%run src/process-dim-armor.py